In [1]:
import sys
import cv2
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer
from fer import FER
import numpy as np

In [2]:
class SmartMirror(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Smart Mirror")
        self.setStyleSheet("background-color: black;")
        self.resize(800, 600)

        # Emotion Detector
        self.detector = FER()

        # Webcam
        self.capture = cv2.VideoCapture(0)

        # UI Elements
        self.image_label = QLabel()
        self.text_label = QLabel()
        self.text_label.setStyleSheet("color: white; font-size: 24px;")
        
        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        layout.addWidget(self.text_label)
        self.setLayout(layout)

        # Timer to update frames
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(100)

    def update_frame(self):
        ret, frame = self.capture.read()
        if not ret:
            return

        # Flip image (mirror effect)
        frame = cv2.flip(frame, 1)

        # Detect emotion
        result = self.detector.detect_emotions(frame)
        message = "🙂 Welcome"

        if result:
            emotions = result[0]["emotions"]
            top_emotion = max(emotions, key=emotions.get)
            message = self.get_message(top_emotion)

            # Draw face box
            (x, y, w, h) = result[0]["box"]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, top_emotion, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Update text
        self.text_label.setText(message)

        # Convert to QImage
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_frame.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)

        # Show in GUI
        self.image_label.setPixmap(QPixmap.fromImage(qt_image))

    def get_message(self, emotion):
        responses = {
            "happy": "😊 You're glowing today!",
            "sad": "💙 It's okay to feel down. Breathe and smile.",
            "angry": "😠 Let's take a moment to calm down.",
            "surprise": "😮 Something unexpected?",
            "fear": "😨 You're safe here. Deep breaths.",
            "disgust": "😖 Step back and center yourself.",
            "neutral": "😐 Stay calm and focused."
        }
        return responses.get(emotion, "🙂 How are you feeling today?")

    def closeEvent(self, event):
        self.capture.release()


In [3]:
app = QApplication(sys.argv)
mirror = SmartMirror()
mirror.showFullScreen()  # Full screen for mirror effect
sys.exit(app.exec_())

SystemExit: 0

  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)

